### Imports

In [ ]:
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms, utils
from torch.utils.data import TensorDataset, DataLoader
import torch.backends.cudnn as cudnn
import time
from pathlib import Path
import os

from art.attacks.evasion import ElasticNet
from art.attacks.evasion import ProjectedGradientDescentPyTorch as PGDAttack
from art.estimators.classification import PyTorchClassifier
from art.utils import load_cifar10

from QuantResNet18 import resnet20 as QuantResNet
from _utils import train, test


%matplotlib inline
%config InlineBackend.figure_format='retina'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
use_cuda = True

In [ ]:
(x_train, y_train), (x_test, y_test), min_, max_ = load_cifar10()

x_train = np.swapaxes(x_train, 1, 3).astype(np.float32)
x_test = np.swapaxes(x_test, 1, 3).astype(np.float32)

train_dataset = TensorDataset(torch.Tensor(x_train), torch.Tensor(y_train))
train_dataloader = DataLoader(train_dataset, batch_size=128)

test_dataset = TensorDataset(torch.Tensor(x_test), torch.Tensor(y_test))
test_dataloader = DataLoader(test_dataset, batch_size=1000)
test_dataloader_single =  DataLoader(test_dataset, batch_size=1)

### Pretrained quantized model

In [ ]:
# Change the self.bits argument in QuantModules
PATH = './models/resnet18_3bit.pth'

# Define what device we are using
print("CUDA Available: ",torch.cuda.is_available())
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

# Initialize models.
net = QuantResNet().to(device)

# Load pre-trained model
net.load_state_dict(torch.load(PATH, map_location='cpu'))

# Load loss and optimiser
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1,
                      momentum=0.9, weight_decay=5e-4)

# Make a classifier wrapper!
classifier = PyTorchClassifier(
    model=net,
    clip_values=(min_, max_),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(3, 32, 32),
    nb_classes=10,
)


# Test model
predictions = classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on benign test examples: {} %".format(accuracy * 100))

### Elastic Net attack

In [ ]:
def EAD_L1(classifier, x_test):
    attack = ElasticNet(classifier=classifier,
                        learning_rate=1e-2,
                        binary_search_steps=5,
                        max_iter=10,
                        beta=1e-3,
                        initial_const=1e-3,
                        decision_rule="L1")
    x_test_adv = attack.generate(x=x_test)
    return x_test_adv

In [ ]:
# create adversarial samples on pruned models and attack pruned models 

n = 1000
x_test_adv_l1 = EAD_L1(classifier, x_test[:n])
predictions = classifier.predict(x_test_adv_l1)
acc = 100*(np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test[:n], axis=1)) / n)
    
print(' Accuracy:', acc)

### Projected Gradient Descent attack

In [ ]:
def PGD(classifier, x_test, norm, eps, alpha):
    attack = PGDAttack(estimator=classifier,
                       norm=norm,
                       eps=eps,
                       eps_step=alpha,
                       max_iter=100,
                       verbose=False)
    x_test_adv = attack.generate(x=x_test)
    return x_test_adv

In [ ]:
# L2

n = 1000
x_test_adv_l2 = PGD(classifier, x_test[:n], norm=2, eps=400/255., alpha=200/255.)
predictions = classifier.predict(x_test_adv_l2)
acc = 100*(np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test[:n], axis=1)) / n)
    
print(' Accuracy:', acc)

In [ ]:
# Linf
n = 1000
x_test_adv_linf = PGD(classifier, x_test[:n], norm=np.inf, eps=4/255., alpha=2/255.)
predictions = classifier.predict(x_test_adv_linf)
acc = 100*(np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test[:n], axis=1)) / n)
    
print(' Accuracy:', acc)